get range of different arguments from the muse api documentation page
info saved to this folder https://drive.google.com/drive/folders/1nR_eUsXeQSBVm5lMa--VZWkrotp-6o0s?usp=share_link

used in the requests in the api to csv notebook: https://colab.research.google.com/drive/1rUwCaekTr7JhBM_a9gJ4sru0WAL5ifGP

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random

from google.colab import files
from datetime import datetime
from IPython.display import display

def unqFileName(base):
  now = datetime.now()
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d%m%Y_%H%M%S")
  return base + "_" + dt_string + ".csv"

def downloadOrDisplay(file, df, head=False, both=False):  
  download = ""
  while both==False and download != "Y" and download != "N":
    download = input("download? [y/n] --> ").upper()
  if both or download == "Y":
    fNm = unqFileName(file)
    df.to_csv(fNm, encoding='utf-8') 
    files.download(fNm)
  if both or download != "Y":
    if head:
      display(df.head())
    else:
      display(df.sample(30))



In [ ]:
url = "https://www.themuse.com/developers/api/v2"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
exps = soup.findAll('div', attrs={"class" : "form-group expansion"})
locs = [tag.attrs['id'] for tag in exps[3].findAll('input')] #list of "city, area" (where area is two letter state or a country)
categs = [tag.attrs['id'] for tag in exps[1].findAll('input')] #list of "indsutry"

In [ ]:
locDict = {}
usLocs = {"city":[], "state":[]}

for loc in locs:
  loc2 = loc.split(", ", 1)
  if len(loc2) == 1: #Canada, Flexible / Remote, Hong Kong, Singapore, United States
    loc2 = [""] + loc2
  city, area = loc2

  if len(area) == 2 and area.isupper():
    usLocs["city"].append(city)
    usLocs["state"].append(area)

  # if city == "":
  #   print(loc)
  if locDict.get(area,0) == 0:
    locDict[area] = []
  locDict[area].append(city)

locDfDct = {'area':[]}
maxCityI = 0

for i, area in enumerate(locDict.keys()): #locDict = area -> [city, city, city]
  locDfDct["area"].append(area)

  for cityi in range( max(  len(locDict[area]), maxCityI+1 )  ):
    city = locDict[area][cityi] if cityi < len(locDict[area]) else pd.NA        #fill out up to max if the number of cities is < number of city columns so far
    if locDfDct.get('city'+str(cityi), 0) == 0:
      locDfDct['city'+str(cityi)] = [pd.NA for i in range(len(locDfDct['area'])-1)] #NA for all previous areas for this city i
      maxCityI = cityi
    locDfDct['city'+str(cityi)].append( city )


locDf = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in locDfDct.items() ]))
usLocDf = pd.DataFrame(usLocs)


downloadOrDisplay("museAPILocations", locDf, both=True)
downloadOrDisplay("museAPIUSLocations", usLocDf, both=True)

#downloadOrDisplay("museAPILocations", locDf.iloc[27], both=True)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,area,city0,city1,city2,city3,city4,city5,city6,city7,city8,...,city2326,city2327,city2328,city2329,city2330,city2331,city2332,city2333,city2334,city2335
138,Eritrea,Asmara,Assab,Barentu,Keren,Massawa,Mendefera,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
119,South Korea,Andong,Ansan-si,Anseong,Asan,Beolgyo,Busan,Changnyeong,Changsu,Changwon,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,Russia,Abakan,Abaza,Abdulino,Abinsk,Achinsk,Achkhoy-Martan,Adler,Afipskiy,Agidel’,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
156,Gambia,Bakau,Banjul,Brikama,Farafenni,Lamin,Sukuta,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
141,Kyrgyzstan,At-Bashi,Balykchy,Bazar-Korgon,Bishkek,Cholpon-Ata,Iradan,Isfana,Jalal-Abad,Kant,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
208,Slovenia,Celje,Koper,Kranj,Ljubljana,Maribor,Novo Mesto,Ptuj,Trbovlje,Velenje,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
131,Trinidad and Tobago,Arima,Chaguanas,Laventille,Marabella,Mon Repos,Paradise,Point Fortin,Port of Spain,Rio Claro,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
71,SC,Aiken,Anderson,Bluffton,Charleston,Clemson,Clinton,Columbia,Conway,Florence,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
173,Belize,Belize City,Belmopan,Orange Walk,San Ignacio,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
265,Comoros,Moroni,Moutsamoudou,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,city,state
1179,Marquette,MI
1286,Morrisville,NC
659,Farmington Hills,MI
2179,Yakima,WA
1203,McLean,VA
1584,Preston,CT
1103,Littlestown,PA
209,Boxborough,MA
850,Hayward,CA
1906,Stratford,CT


In [ ]:
catDf = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in {"Category":categs}.items() ]))

downloadOrDisplay("museAPICategories", catDf, both=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Category
22,Farming and Outdoors
26,Human Resources and Recruitment
6,Animal Care
24,Healthcare
9,Cleaning and Facilities
51,Social Services
27,"Installation, Maintenance, and Repairs"
52,Software Engineer
40,Physical Assistant
53,Software Engineering


In [ ]:
# Install the SQLAlchemy library if it is not installed
!sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
!sudo pip3 install -U sql_magic > /dev/null
!pip install psycopg2-binary > /dev/null

In [ ]:
from sqlalchemy import create_engine

In [ ]:
# Connect to the MySQL, but without selecting a database
conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset={encoding}'.format(
    user='team', 
    password='tbmFmVOE6gg=', 
    host = 'jsedocc7.scrc.nyu.edu', 
    port     = 3306, 
    encoding = 'utf8',
    db = 'Team'
)
engine = create_engine(conn_string)

In [ ]:
import pandas as pd
from google.colab import files
import io

In [ ]:
#locDf.to_sql('TheMuseAllLocs', if_exists='replace', con=engine) #could do append or fail
usLocDf.to_sql('TheMuseUSLocs', if_exists='replace', con=engine)
catDf.to_sql('TheMuseAllCats', if_exists='replace', con=engine)

In [ ]:
#used in the requests in the api to csv notebook: https://colab.research.google.com/drive/1rUwCaekTr7JhBM_a9gJ4sru0WAL5ifGP

j = 0
each = 548

while True:
  reqStr = "https://www.themuse.com/api/public/jobs?"
  #2191 cities in usLocs
  if (j+each) <= len(usLocs):
    usSubSet = usLocs[j:j+each]
  else:
    usSubSet = usLocs[j:]
  print(j, j+each)

  for i, loc in enumerate(usSubSet): #[:598] works, [1600:] does not work, [1700:] works i = 598, 599, 600 cause problems
    city, area = loc.split(", ")
    reqStr += "location=" + city + "%2C%20" + area + "&"
  reqStr += "page=1"
  print(reqStr)
  resp = requests.get(reqStr)
  res0 = resp.json()['results'][0]
  #print(res0)

  if (j+each) > len(usLocs):
    break
  j += each


0 548
https://www.themuse.com/api/public/jobs?location=Aberdeen%2C%20MD&location=Aberdeen%2C%20SD&location=Aberdeen%2C%20WA&location=Abilene%2C%20TX&location=Ada%2C%20OK&location=Addison%2C%20IL&location=Addison%2C%20TX&location=Adelanto%2C%20CA&location=Adrian%2C%20MI&location=Agawam%2C%20MA&location=Agoura Hills%2C%20CA&location=Ahwatukee%2C%20AZ&location=Aiken%2C%20SC&location=Akron%2C%20OH&location=Alameda%2C%20CA&location=Alamogordo%2C%20NM&location=Albany%2C%20GA&location=Albany%2C%20NY&location=Albany%2C%20OR&location=Albemarle%2C%20NC&location=Albert Lea%2C%20MN&location=Albuquerque%2C%20NM&location=Alcoa%2C%20TN&location=Alexandria%2C%20LA&location=Alexandria%2C%20VA&location=Alice%2C%20TX&location=Allen Park%2C%20MI&location=Allentown%2C%20PA&location=Allen%2C%20TX&location=Alliance%2C%20OH&location=Alma%2C%20MI&location=Alpena%2C%20MI&location=Alpharetta%2C%20GA&location=Altamont%2C%20OR&location=Alton%2C%20IL&location=Altoona%2C%20PA&location=Altus%2C%20OK&location=Amarillo